In [ ]:
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
def get_stock_data(symbol, start_dt, end_dt):
    stock_data = yf.download(symbol, start=start_dt, end=end_dt)
    return stock_data

In [ ]:
def ema(data, duration):
    return data['Close'].ewm(span=duration, adjust=False).mean()

In [ ]:
def objective_function(weights, cov_matrix, expected_returns, risk_aversion):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    portfolio_return = np.dot(expected_returns, weights)
    return -1 * (portfolio_return - risk_aversion * portfolio_variance)

In [ ]:
def constraint(weights):
    return np.sum(weights) - 1.0

In [ ]:
def maximize_objective(expected_returns, covariance_matrix, risk_aversion):
    num_assets = len(expected_returns)
    initial_weights = np.ones((num_assets,)) / num_assets
    bounds = tuple((0, 1) for asset in range(num_assets))
    constraints = ({'type': 'eq', 'fun': constraint})
    result = minimize(objective_function, initial_weights, args=(covariance_matrix, expected_returns.flatten(), risk_aversion), method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

In [ ]:
def select_portfolio(symbols, start_dt, end_dt):
    portfolios = []

    current_date = datetime.strptime(start_dt, '%Y-%m-%d')
    end_date = datetime.strptime(end_dt, '%Y-%m-%d')

    while current_date <= end_date:
        current_month_start = current_date.replace(day=1)
        next_month_start = (current_date + timedelta(days=32)).replace(day=1)  # Move to the next month

        # Fetch stock data for the current month
        stock_data_dict = {}
        for symbol in symbols:
            stock_data = get_stock_data(symbol, current_month_start, next_month_start)
            if stock_data is not None and not stock_data.empty:
                stock_data_dict[symbol] = stock_data

        # Calculate EMAs and select stocks for the portfolio
        portfolio = []
        for symbol, stock_data in stock_data_dict.items():
            monthly_data = stock_data.resample('D').first()

            ema_9 = ema(monthly_data, 9).iloc[-1]
            ema_21 = ema(monthly_data, 21).iloc[-1]

            if ema_9 > ema_21:
              portfolio.append(symbol)

        # Append the portfolio to the list
        portfolios.append((current_month_start, portfolio))

        # Move to the next month
        current_date = next_month_start

    return portfolios

In [ ]:
def get_optimal_weights(selected_portfolio, start_dt, end_dt, risk_aversion):
    optimal_weights_list = []

    for date, portfolio in selected_portfolio:
        stock_data_dict = {}
        for symbol in portfolio:
            stock_data = get_stock_data(symbol, start_dt, end_dt)
            if stock_data is not None and not stock_data.empty:
                stock_data_dict[symbol] = stock_data

        if not stock_data_dict:
            continue  # Skip if no valid stock data

        returns_matrix = np.vstack([np.log(stock_data_dict[symbol]['Close'] / stock_data_dict[symbol]['Close'].shift(1)).dropna().values for symbol in portfolio])

        if returns_matrix.size == 0:
            continue  # Skip if returns_matrix is empty

        expected_returns = np.mean(returns_matrix, axis=1)
        covariance_matrix = np.cov(returns_matrix)
        expected_returns = expected_returns.reshape(-1, 1)
        covariance_matrix = covariance_matrix + np.eye(len(expected_returns)) * 1e-6

        optimal_weights = maximize_objective(expected_returns, covariance_matrix, risk_aversion)
        optimal_weights_list.append((date, portfolio, optimal_weights))

    return optimal_weights_list

In [ ]:
def plot_pie_chart(date, portfolio, weights):
    plt.figure(figsize=(8, 8))
    plt.pie(weights, labels=portfolio, autopct='%1.1f%%', startangle=140)
    plt.title(f'Portfolio Weightage for {date.strftime("%Y-%m-%d")}')
    plt.show()

In [ ]:
def calculate_cagr(optimal_weights_list):  # Removed the portfolio_periods argument
    cumulative_returns = []

    for date, portfolio, weights in optimal_weights_list:
        # Access start_dt and end_dt directly from the tuple
        start_dt, end_dt = date, date + timedelta(days=32)

        # Fetch stock data for the current portfolio period
        stock_data_dict = {}
        for symbol in portfolio:
            if symbol not in stock_data_dict:
                stock_data_dict[symbol] = get_stock_data(symbol, start_dt, end_dt)

        if not stock_data_dict:
            continue  # Skip if no valid stock data

        daily_returns = np.vstack([np.log(stock_data_dict[symbol]['Close'] / stock_data_dict[symbol]['Close'].shift(1)).dropna().values for symbol in portfolio])

        if daily_returns.size == 0:
            continue  # Skip if returns_matrix is empty

        # Calculate time-weighted returns for each portfolio
        weighted_returns = np.average(daily_returns, axis=0, weights=weights)  # Use weighted average
        cumulative_return = np.cumprod(1 + weighted_returns)[-1] - 1
        cumulative_returns.append(cumulative_return)

    # Compute the overall CAGR using the geometric mean of cumulative returns
    net_cagr = (np.prod(1 + np.array(cumulative_returns)) ** (1 / len(cumulative_returns))) - 1

    return net_cagr

In [ ]:
def plot_pie_chart(date, portfolio, weights):
    plt.figure(figsize=(8, 8))
    plt.pie(weights, labels=portfolio, autopct='%1.1f%%', startangle=140)
    plt.title(f'Portfolio Weightage for {date.strftime("%Y-%m-%d")}')
    plt.show()

ecommerce_symbols = ['AMZN', 'BABA', 'EBAY', 'MELI', 'PDD', 'RKUNF', 'SHOP']
banks_symbols = ['SBIN.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'AXISBANK.NS', 'KOTAKBANK.NS', 'PNB.NS', 'BANKBARODA.NS']
automobiles_symbols = ['TSLA', 'TM', 'GM', 'F', 'STLA', 'HMC', 'RACE']
aviation_symbols = ['BA', 'LMT', 'HON', 'ESLT', 'RTX', 'NOC', 'TDG']
oil_symbols = ['SLB', 'CVX', 'EQNR', 'XOM', 'COP', 'TTE', 'BP']
start_dt = '2019-05-01'
end_dt = '2022-05-01'

i = int(input(" E-Commerce: 1 \n Banks: 2 \n Automobiles: 3 \n Aviation: 4 \n Oil: 5 \n Enter the sector you wish to use: "))
if i == 1:
    symbols = ecommerce_symbols
elif i ==2:
    symbols = banks_symbols
elif i == 3:
    symbols = automobiles_symbols
elif i == 4:
    symbols = aviation_symbols
elif i == 5:
    symbols = oil_symbols

resulting_portfolios = select_portfolio(symbols, start_dt, end_dt)

risk_aversion_parameter = float(input("Enter the risk aversion parameter: "))

In [ ]:
optimal_weights_list = get_optimal_weights(resulting_portfolios, start_dt, end_dt, risk_aversion_parameter)

In [ ]:
print("Optimized Portfolios:")

for date, portfolio, weights in optimal_weights_list:
    print(f"\nPortfolio for {date.strftime('%Y-%m-%d')}: {portfolio}")
    print("\n")
    print(f"Optimal Weights: {weights}")
    print("\n")
    plot_pie_chart(date, portfolio, weights)
    print("\n")

In [ ]:
portfolio_periods = []
for date, portfolio, weights in optimal_weights_list:
    portfolio_periods.append((date, portfolio, weights))

In [ ]:
net_cagr = calculate_cagr(optimal_weights_list)
if net_cagr is not None:
    print("\n")
    print(f"Net CAGR for the entire duration: {net_cagr * 100:.2f}%")
else:
    print("Net CAGR: Not Calculated (Incomplete Data)")